In [ ]:
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
from fenics_adjoint import *
import moola
from SimpleBrainMesh import generate_doughnut_mesh
from DarcySolver import solve_darcy_on_doughnut_brain

T = 5.0           # final time
num_steps = 100    # number of time steps
dt = T/ num_steps

N = 20 # resolution
brain_radius = 0.1  # brain circle Radius
ventricle_radius = brain_radius/3
aqueduct_width = brain_radius/5

kappa = 5*1e-8    # permeability 15*(1e-9)**2
visc = 0.8*1e-3     # viscocity 
K = kappa/visc      # hydraulic conductivity

a_0 = 0.6
a_1 = 1.5
f_0 = 1.0/(60/15)
f_1 = 1.0/(60/70)
c = 7 
mmHg2Pa = 133.32

In [ ]:
# create mesh

mesh = Mesh(generate_doughnut_mesh(brain_radius, ventricle_radius, aqueduct_width, N))

probe_points = [
                Point(ventricle_radius + 0.25*(brain_radius - ventricle_radius), 0), 
                Point(ventricle_radius + 0.75*(brain_radius - ventricle_radius), 0), 
                ]

x_coords = np.linspace(ventricle_radius, brain_radius, 1000)
slice_points = [Point(x, 0.0) for x in x_coords]

p_obs = Expression("""c*mmHg2Pa*(a_0 * sin(2 *pi * f_0*t) + a_1 *sin(2*pi*f_1*t))
                   *( sqrt( x[0]*x[0] + x[1]*x[1] ) - R_ventricle ) """,
                    a_0=a_0, a_1=a_0, f_0=f_0, f_1=f_1, t = 0,c=c,
                    mmHg2Pa=mmHg2Pa, R_ventricle=ventricle_radius,
                    degree = 2)

control_space = FunctionSpace(mesh, "CG", 1)

In [ ]:
ctrls = [Function(control_space, name="control") for i in range(num_steps)]

control = [Control(c) for c in ctrls]

results_init = solve_darcy_on_doughnut_brain(mesh, ctrls, T, num_steps, K=K,
                                             probe_points=probe_points,
                                             slice_points=slice_points,
                                             dirichlet_boundary_skull=p_obs,
                                             dirichlet_boundary_ventricle=p_obs, p_obs=p_obs,
                                             file_name="../results/p_init_distr_ctrl_darcy.xdmf")

rf = ReducedFunctional(results_init["J"], control)
f_moola = moola.DolfinPrimalVectorSet([moola.DolfinPrimalVector(c, inner_product="L2") for c in ctrls])
#solver = moola.NewtonCG(problem, f_moola, options={'gtol': 1e-9,
#                                                   'maxiter': 20,
#                                                   'display': 3,
#                                                   'ncg_hesstol': 0})

solver = moola.BFGS(problem, f_moola, options={'jtol': 0,
                                               'gtol': 1e-9,
                                               'Hinit': "default",
                                               'maxiter': 20,
                                               'mem_lim': 10})
sol = solver.solve()
opt_ctrls = sol['control'].data


In [ ]:
xdmf_file = XDMFFile("../results/f_opt_distr_ctrl_darcy.xdmf")
xdmf_file.parameters["flush_output"] = True
xdmf_file.parameters["functions_share_mesh"] = True
xdmf_file.parameters["rewrite_function_mesh"] = False

for n in range(num_steps):
    opt_ctrls[n].rename("f","control")
    xdmf_file.write(opt_ctrls[n], n*T / num_steps)
    
results_opt = solve_darcy_on_doughnut_brain(mesh, opt_ctrls, T, num_steps,
                                            probe_points=probe_points,
                                            slice_points=slice_points,
                                            dirichlet_boundary_skull=p_obs,
                                            dirichlet_boundary_ventricle=p_obs, p_obs=p_obs,
                                            file_name="../results/p_opt_distr_ctrl_darcy.xdmf")


In [ ]:
plt.figure(figsize=(8,8))
cont = plot(mesh)
for i, point in enumerate(probe_points):
    plt.scatter(point.array()[0], point.array()[1],
                marker=".", s= 200, c="red")
    plt.annotate( "$P_{}$".format(i), point.array()[0:2], size=20)

In [ ]:
num_points = len(probe_points)
for i in range(num_points):
    fig,ax=plt.subplots(figsize=(8,8))
    plt.plot(np.linspace(0,T, num_steps),
             results_init["probe_point_obs_data"][:,i],
             label="observed $P_{}$".format(i),
            )
    plt.plot(np.linspace(0,T, num_steps), 
             results_init["probe_point_data"][:,i], 
             label="initial $P_{}$".format(i)
            )

    plt.plot(np.linspace(0,T, num_steps), 
             results_opt["probe_point_data"][:,i], 
             "*",label="optimized $P_{}$".format(i)
            )
    plt.xlabel("t in s")
    plt.ylabel("p in Pa")
    plt.legend()
    ax.twinx()

    plt.plot(np.linspace(0,T, num_steps), 
             results_opt["probe_point_f_data"][:,i], 
             ls="dotted", c="red",
             label="optimized force $P_{}$".format(i)
            )
    plt.ylabel("f in N")
    plt.grid()
    plt.legend()
    plt.title("$P_{}$".format(i))

In [ ]:
p_init = results_init["slice"]
p_opt = results_opt["slice"]
p_target = results_init["target_slice"]
f_opt = results_opt["f_slice"]
times = [1,2,3,4,5]
idx = []
for i in range(num_steps):
    if np.isclose((i +1)*dt, times).any():
        idx.append(i)
        

for i in idx:
    fig,ax=plt.subplots(figsize=(8,8))
    plt.title("t = {}".format((i+1)*dt))
    plt.plot(x_coords, p_init[i,:], 
             label="p_init")
    plt.plot(x_coords, p_opt[i,:], 
             "*",label="p_opt")
    plt.plot(x_coords, p_target[i,:], 
             label="p_target")
    plt.legend(loc="upper left")
    plt.xlabel("x in m")
    plt.ylabel("p in Pa")
    ax.twinx()
    plt.plot(x_coords, 
             f_opt[i,:], 
             ls="dotted", c="red",
             label="optimized force")
    
    plt.ylabel("f in N")
    plt.grid()
    plt.legend(loc="upper center")

In [ ]:
f_max = np.max([f_opt.vector().max() for f_opt in opt_ctrls])
f_min = np.min([f_opt.vector().min() for f_opt in opt_ctrls])

for i in [0,2,4,6,8, 10, 12]:
    plt.figure()
    c = plot(opt_ctrls[i],vmin=f_min, vmax=f_max)
    plt.colorbar(c)
    plt.title("t = {}".format(i*T/num_steps))

